In [1]:
import os
import glob

import numpy as np


from medpy.metric import binary

from PIL import Image

import imageio

In [2]:
# task = '_betnorm' # _gmnorm _ventnorm _wmnorm _betnorm _csfnorm
# gt_dir = '/data/ychan019/experiments/radiaide/radiaide/code/inference/test_kcl_p2p_seg_alt' + task + '/gt/'
# syn_dir = '/data/ychan019/experiments/radiaide/radiaide/code/inference/test_kcl_p2p_seg_alt' + task + '/mask/' # sub-0032_20.png
task = 'gm_alt'
gt_dir = '/data/data_repo/neuro_img/anat_brain_img/datasets/CERMEP-IDB-MRXFDG/processed/yihao/labels_new_test/' + task
syn_dir = '/data/ychan019/experiments/radiaide/Pytorch-UNet-master/data/mask_' + task

In [3]:
all_filepaths_gt = sorted(glob.glob(gt_dir + "/*.*"))
all_filepaths_syn = sorted(glob.glob(syn_dir + "/*.*"))

In [4]:
from collections import defaultdict

prev_subject_id = 'sub-0028'
hist_dc = defaultdict(list)
hist_jc = defaultdict(list)

fp = []
missed = []

for subj_path_gt, subj_path_syn in zip(all_filepaths_gt, all_filepaths_syn):
    
    # gt = np.array(Image.open(subj_path_gt).convert("L"))
    gt = np.array(imageio.imread(subj_path_gt)) # filename
    syn = np.array(Image.open(subj_path_syn).convert("L"))
    
    if np.sum(gt) == 0: # and np.sum(syn) == 0: # if no incl syn this will miss out on false positive
        if np.sum(syn) != 0:
            fp.append((subj_path_gt, subj_path_syn))
        continue
    
    cur_subject_id_full = subj_path_syn.split('/')[-1]
    cur_subject_id = cur_subject_id_full.split('_')[0]
    
    dc = binary.dc(gt, syn) # https://loli.github.io/medpy/generated/medpy.metric.binary.dc.html#medpy.metric.binary.dc
    jc = binary.jc(gt, syn) # https://loli.github.io/medpy/generated/medpy.metric.binary.jc.html#medpy.metric.binary.jc
    print(cur_subject_id_full, dc, jc)
    
    if cur_subject_id != prev_subject_id:
        print()
    
    prev_subject_id = cur_subject_id
    
    if dc != 0 or jc != 0:
        hist_dc[cur_subject_id].append(dc)
        hist_jc[cur_subject_id].append(jc)
    else:
        missed.append((subj_path_gt, subj_path_syn))

sub-0028_100.png 0.6916322074060307 0.528622159090909
sub-0028_101.png 0.7048260381593715 0.5441941074523396
sub-0028_102.png 0.6980944725518937 0.5362097770029781
sub-0028_103.png 0.6885214565612157 0.524996356216295
sub-0028_104.png 0.6759182072279704 0.5104807051329531
sub-0028_105.png 0.6629270264806953 0.49580467155491725
sub-0028_106.png 0.649398029916089 0.4808211777417612
sub-0028_107.png 0.6335024273168604 0.4635957209005269
sub-0028_108.png 0.6328718177757927 0.4629206141783731
sub-0028_109.png 0.6264272190028114 0.45605680868838766
sub-0028_110.png 0.6364267530664198 0.46673455532926
sub-0028_111.png 0.6313808515643378 0.46132691646613955
sub-0028_112.png 0.6357763683345079 0.46603529918209213
sub-0028_113.png 0.6313447690416252 0.4612883907947643
sub-0028_114.png 0.6239712530427727 0.4534580069075899
sub-0028_115.png 0.6347998864604031 0.46498669328010644
sub-0028_116.png 0.6432702120421816 0.4741328875888007
sub-0028_117.png 0.6374656907593779 0.46785294611381567
sub-0028_

In [5]:
all_dc = []

for k,v in hist_dc.items():
    
    all_dc.extend(v)
    
    sub_scores = np.array(v)
    print(k, sub_scores.mean(), sub_scores.std(), sub_scores.min(), sub_scores.max())


sub-0028 0.6217687020593279 0.15739232472794468 0.0072992700729927005 0.8830150068212824
sub-0029 0.6380209340588472 0.13896489395401668 0.007759456838021339 0.8785504407443683
sub-0030 0.6509345624854631 0.1390842411444979 0.03307888040712468 0.7984590078781058
sub-0031 0.6717280118126037 0.12568385491467762 0.06796116504854369 0.8814349775784753
sub-0032 0.6074113032931148 0.14026133647236286 0.04020100502512563 0.7600473980448307
sub-0033 0.6030125677121644 0.1496614524944806 0.005277044854881266 0.8913202042304887
sub-0034 0.6613198780260316 0.1542436875784021 0.021447721179624665 0.9002132196162047
sub-0035 0.6420989458688205 0.1310869416900149 0.07624633431085044 0.8902346707040121
sub-0036 0.6350708182652844 0.10748348715289385 0.14418604651162792 0.7837998219760657
sub-0037 0.6561978020697555 0.1573327141230311 0.02717391304347826 0.8697925745162978


In [6]:
all_jc = []

for k,v in hist_jc.items():

    all_jc.extend(v)
    
    sub_scores = np.array(v)
    print(k, sub_scores.mean(), sub_scores.std(), sub_scores.min(), sub_scores.max())


sub-0028 0.46832122925131514 0.15262365310328196 0.003663003663003663 0.7905343511450381
sub-0029 0.4820022178524985 0.13516457828361078 0.0038948393378773127 0.7834061135371179
sub-0030 0.4956042469841546 0.12837775328442116 0.016817593790426907 0.6645291447510627
sub-0031 0.5173166344919802 0.12455091970027712 0.035175879396984924 0.7880051314945478
sub-0032 0.44841403084444265 0.12275880451797123 0.020512820512820513 0.6129648801465318
sub-0033 0.446366446364289 0.13978428641165122 0.0026455026455026454 0.8039473684210526
sub-0034 0.5107604084720402 0.14786335050485505 0.01084010840108401 0.8185343156262117
sub-0035 0.48512933836158656 0.1290523757899855 0.039634146341463415 0.8021828103683493
sub-0036 0.47326895245435735 0.10238538048530028 0.07769423558897243 0.6444661299503944
sub-0037 0.5048807782328587 0.14370996534320735 0.013774104683195593 0.7695866748920419


In [7]:
all_dc = np.array(all_dc)
all_jc = np.array(all_jc)

In [8]:
all_dc.mean(), all_dc.std()

(0.6389680675172446, 0.14280714524626759)

In [9]:
all_jc.mean(), all_jc.std()

(0.4834876539658901, 0.13562030071504658)

In [10]:
len(fp)

10

In [11]:
len(missed)

68

In [12]:
gt

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [13]:
syn

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [14]:
np.unique(gt)

array([0, 1], dtype=uint8)

In [15]:
np.unique(syn)

array([  0, 127], dtype=uint8)

In [16]:
np.sum(gt), np.sum(syn)/127

(9642, 13657.0)